In [1]:
import pandas as pd

In [2]:
url = 'https://raw.githubusercontent.com/dhimmel/drugbank/3e87872db5fca5ac427ce27464ab945c0ceb4ec6/data/mapping/pubchem.tsv'
drugbank_map_df = pd.read_table(url)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


In [3]:
drugbank_map_df.head()

,drugbank_id,pubchem_id
0,DB00014,11980055
1,DB00014,11981235
2,DB00014,11982741
3,DB00014,16052011
4,DB00014,23581804


In [4]:
from rdflib import Graph, URIRef, Literal, RDF, ConjunctiveGraph
def to_rdf(g, df, column_types, row_uri):
    """
    Parameters
    ----------
    g : input rdflib.Graph  
    df: DataFrame to be converted into RDF Graph
    column_types: dictonary of column and its type, type can be URI or Literal
    row_uri: rdf:type value for row index, should be URI
    Returns
    -------
    g: rdflib.Graph generated from DataFrame object
    """
    
    for (index, series) in df.iterrows():
        g.add((URIRef(index), RDF.type, URIRef(row_uri)) )
        for (column, value) in series.iteritems():
            if column_types[column] == 'URI':
                g.add((URIRef(index), URIRef(column), URIRef(value)))
            else:
                g.add((URIRef(index), URIRef(column), Literal(value)))
                
    return g

In [5]:
drugbank_map_df['drugbank_id'] = drugbank_map_df['drugbank_id'].map(lambda x: 'http://bio2rdf.org/drugbank:'+str(x))

In [6]:
drugbank_map_df['pubchem_id'] = drugbank_map_df['pubchem_id'].map(lambda x: 'http://bio2rdf.org/pubchem.compound:'+str(x))

In [7]:
drugbank_map_df.head()

,drugbank_id,pubchem_id
0,http://bio2rdf.org/drugbank:DB00014,http://bio2rdf.org/pubchem.compound:11980055
1,http://bio2rdf.org/drugbank:DB00014,http://bio2rdf.org/pubchem.compound:11981235
2,http://bio2rdf.org/drugbank:DB00014,http://bio2rdf.org/pubchem.compound:11982741
3,http://bio2rdf.org/drugbank:DB00014,http://bio2rdf.org/pubchem.compound:16052011
4,http://bio2rdf.org/drugbank:DB00014,http://bio2rdf.org/pubchem.compound:23581804


In [8]:
drugbank_map_df = drugbank_map_df.set_index('drugbank_id', drop=True)

In [9]:
drugbank_map_df.rename(columns={'pubchem_id':'http://bio2rdf.org/openpredict_vocabulary:x-pubchemcompound'},inplace=True)

In [10]:
column_types ={'http://bio2rdf.org/openpredict_vocabulary:x-pubchemcompound':'URI'}

g =  ConjunctiveGraph(identifier = URIRef('http://bio2rdf.org/openpredict_resource:fairworkflows.dataset.openpredict.pubhem.R1'))     

g=  to_rdf(g, drugbank_map_df, column_types, 'http://bio2rdf.org/drugbank:Drug' )

In [11]:
g.serialize('data/rdf/pubchem_mapping.nq', format='nquads')